In [53]:
import torch
import numpy as np
import pandas as pd
import math
from classifier.models.cnn1d import PositionClassifier
import matplotlib.pyplot as plt

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def load(fn, device):
    d = torch.load(fn, map_location=device)
    
    return d['model'], d['config']

def test(x, output_size, model_dict, train_config):
    model = PositionClassifier(4, output_size).to(device)
    model.load_state_dict(model_dict)
    model.eval()

    detect = []
    for x_i in x:
        with torch.no_grad():
            y_hat = model(x_i.unsqueeze(0))
            detect.append(torch.argmax(y_hat, dim=-1))

    detect = torch.cat(detect, dim=0)

    loss_hist = []  # 오차거리 계산
    for i in range(len(detect)):
        loss_hist.append(abs(i-detect[i])*0.6)  # 보폭 60cm

    return loss_hist


In [55]:
point_n = 245
pattern_n = 5
output_size = point_n - pattern_n
data = pd.read_table("./data/test/s21.txt", sep=" ")
data = pd.DataFrame(data, columns=['X', 'Y', 'Z', 'F'], index=np.linspace(0, len(data)-1, point_n, endpoint=True, dtype='i'))
data = data.reset_index(drop=True)


x = [[] for _ in range(point_n-pattern_n)]
for i in range(point_n-pattern_n):
    x[i] = np.array(data[i:i+pattern_n])
weights = "./weights/model.pt"

x = torch.from_numpy(np.array(x, dtype='f'))

loss_hist = test(x, output_size, *load(weights, device))

print(f"min error distance: {min(loss_hist)}m")
print(f"max error distance: {max(loss_hist)}m")
print(f"avg error distance: {np.mean(loss_hist)}m")

min error distance: 0.0m
max error distance: 1.8000000715255737m
avg error distance: 0.4725000262260437m
